In [2]:
import pandas as pd
import numpy as np
import random
import pyomo.environ as pyo
from pyomo.environ import *
from pyomo.environ import SolverFactory
import yfinance as yf
import matplotlib.pyplot as plt


## INICIAR MODELO

In [27]:
model = pyo.ConcreteModel()

#parametros
capacidade_fabrica = {
    'F1':350,
    'F2':600
}

demanda_cd = {
    'CD1':300,
    'CD2':400,
    'CD3':250
}


# PARA TRANSPORTE, é bom usar TUPLAS ('','')
#De Para CD1,Cd2,Cd3
custo_transporte = {
    ('F1','CD1'):8, ('F1','CD2'):6, ('F1','CD3'):10,
    ('F2','CD1'):9, ('F2','CD2'):12, ('F2','CD3'):13
}

#Definir SET
model.FABRICAS = pyo.Set(initialize=['F1','F2'])
model.CD = pyo.Set(initialize=['CD1','CD2','CD3'])

#Variaveis
model.x = pyo.Var(model.FABRICAS,model.CD, domain=NonNegativeIntegers)

#Restrições de quantidade
def soma_capacidade_fabricas(model, f):
    return sum (model.x[f,c] for c in model.CD) <= capacidade_fabrica[f]
model.soma_capacidade_fabricas = pyo.Constraint(model.FABRICAS, rule=soma_capacidade_fabricas)

def soma_capacidade_cd(model, c):
    return sum(model.x[f,c] for f in model.FABRICAS) >= demanda_cd[c]
model.soma_capacidade_cd= pyo.Constraint(model.CD, rule=soma_capacidade_cd)

#Objetivo
def objetivo1(model):
    
    return sum(model.x[f,c] * custo_transporte[f,c] for f in model.FABRICAS for c in model.CD)

model.objetivo = pyo.Objective(rule=objetivo1, sense=pyo.minimize)



In [28]:
opt = SolverFactory('cplex', executable='C:\\CPLEX_Studio2211\\cplex\\bin\\x64_win64\\cplex.exe')
opt.options['TimeLimit'] = 180 
results = opt.solve(model, tee=True, warmstart=True)
print("-------FIM----------")



Welcome to IBM(R) ILOG(R) CPLEX(R) Interactive Optimizer 22.1.1.0
  with Simplex, Mixed Integer & Barrier Optimizers
5725-A06 5725-A29 5724-Y48 5724-Y49 5724-Y54 5724-Y55 5655-Y21
Copyright IBM Corp. 1988, 2022.  All Rights Reserved.

Type 'help' for a list of available commands.
Type 'help' followed by a command name for more
information on commands.

CPLEX> Logfile 'cplex.log' closed.
Logfile 'C:\Users\DECIV\AppData\Local\Temp\tmp9nl3y17z.cplex.log' open.
CPLEX> New value for time limit in seconds: 180
CPLEX> Problem 'C:\Users\DECIV\AppData\Local\Temp\tmpjkm9hrfv.pyomo.lp' read.
Read time = 0.00 sec. (0.00 ticks)
CPLEX> Warning: no MIP start values read, no MIP start loaded.
MIP start file 'C:\Users\DECIV\AppData\Local\Temp\tmp_0hs8zph.cplex.mst' read.
CPLEX> Problem name         : C:\Users\DECIV\AppData\Local\Temp\tmpjkm9hrfv.pyomo.lp
Objective sense      : Minimize
Variables            :       6  [General Integer: 6]
Objective nonzeros   :       6
Linear constraints   :       5  [

In [30]:
for f in model.FABRICAS:
    for c in model.CD:
        
        print(f"Valor de x{f,c}: ",pyo.value(model.x[f,c]))
        



Valor de x('F1', 'CD1'):  0.0
Valor de x('F1', 'CD2'):  350.0
Valor de x('F1', 'CD3'):  0.0
Valor de x('F2', 'CD1'):  300.0
Valor de x('F2', 'CD2'):  50.0
Valor de x('F2', 'CD3'):  250.0
